# library loading

In [1]:
import pandas as pd
import numpy as np
import re

# function

In [2]:
# 패턴을 이용하여 저장 값을 리스트로 되돌리는 함수
def make_ls(c,i): # 인덱스 리스트와 순서를 받음
    matches = re.findall(r"'(.*?)'", c[i])
    return list(matches)

def clear_text(sentence):  # 목차 내용 정제 함수
    pattern = r'[\w가-힣]+'
    extracted = re.findall(pattern, sentence)
    result = ' '.join(extracted)
    return result
    
def delete_text(sentence):
    pattern = r'u3000|u000|u00|uf0ab|u2005|'#목차 더보기'
    result = re.sub(pattern, '', sentence)
    return result
    
def split_pn(line,e): # 페이지 분리 함수
    a = re.sub(r'[^(X|IX|IV|V?I|Ⅱ|Ⅲ)]','', line) # 로마자인지 구분하기 위함
    if line.split(' ')[-1].isnumeric() == True and a == '':
        page = line.split(' ')[-1]  
        line = ' '.join(line.split(' ')[:-1])
        k = 0
        e = etc_check(e,page,k)
    else:
        page = np.NaN
        k = 'pn'
        e = etc_check(e,k)
    
    return line, page, e

def plus_pandan(pd_l):
    if '총' in pd_l and '각' in pd_l:
        i = pd_l.index('총')
        pd_l[i] = ['총', '각']
        pd_l.remove('각')
    elif '총론' in pd_l and '각론' in pd_l:
        i = pd_l.index('총론')
        pd_l[i] = ['총론', '각론']
        pd_l.remove('각론')
#     elif '총론' in pd_l and '각론' in pd_l and '부록' in pd_l:
#         i = pd_l.index('총론')
#         pd_l[i] = ['총론', '각론', '부록']
    elif '형법총론' in pd_l and '형법각론' in pd_l:
        i = pd_l.index('형법총론')
        pd_l[i] = ['형법총론', '형법각론']
        pd_l.remove('형법각론')
    elif '헌법총론' in pd_l and '헌법각론' in pd_l:
        i = pd_l.index('헌법총론')
        pd_l[i] = ['헌법총론', '헌법각론']
        pd_l.remove('헌법각론')
    else:
        pass
    return pd_l


In [3]:
def classification_list2(text, ex): # 판별 요소 설정 함수
    pd_l,ck_l = [],[]
    for idx, line in enumerate(text):
        a = re.sub(r'[^(X|IX|IV|V?I|Ⅱ|Ⅲ)]','', line)  # r'(X|IX|IV|V?I|Ⅱ|Ⅲ) 이걸로 해보기
        # 숫자 판별
        if line[0].isnumeric() == True: 
            if a != '' and 'roma' not in pd_l: # 로마 숫자
                ck_l.append('roma') # 위치 저장
                pd_l.append('roma')
            elif a != '' and 'roma' in pd_l: # 로마 숫자
                ck_l.append('roma') # 위치 저장
            elif a == '' and line[1] in ['부', '절','편','강'] and 'num' not in pd_l :
                ck_l.append('num')
                pd_l.append('num') # 분류랑 숫자 결합
            elif a == '' and line[1] in ['부', '절','편','강'] and 'num' in pd_l:
                ck_l.append('num')
            elif a == '' and line[1] not in ['부', '절','편','강'] and 'ara' not in pd_l :
                ck_l.append('ara')
                pd_l.append('ara') # 분류랑 숫자 결합
            elif a == '' and line[1] not in ['부', '절','편','강'] and 'ara' in pd_l:
                ck_l.append('ara')
        elif line[0].isnumeric() != True:   # 포함되는 리스트
            t = re.sub(r'[^a-zA-Z가-힣]','',line.lower())  # 기록 시에는 line을 기록해야 함
            b = ex.match(t)
            if b and b.group() not in pd_l:
                pd_l.append(b.group())
                ck_l.append(b.group())
            elif b and b.group() in pd_l:
                ck_l.append(b.group())
                pass
            else:
                pd_l.append('check')
                
    return pd_l, ck_l

In [4]:
def clear_idx(isbn, tit, text, i, ex):
    code = isbn[i] # isbn, main_title
    title = tit[i]
#     d1,d2,d3,d4 = 0,0,0,0  # 뎁스 채울 변수(초기화)
    if '목차 더보기' in text: # 잘못 들어간 단어 삭제(틀 전체로 접근하다 보니 같이 달려옴)
        text.remove('목차 더보기')
    text = [clear_text(r) for r in text] # 이상한 문자 제거
    text = [delete_text(r) for r in text]
    pandan,ck_l = classification_list2(text,ex)
    print('iii')
    e = pandan_sequence(pandan, ck_l)
    print(e)
    pandan = plus_pandan(pandan) 
    if 'check' in pandan:  # 분류구분자에 예외 표현이 존재
        isbn_ls.append(code)
        main_title.append(title)
        etc.append('check')
#         print('check')
        for ls in [depth_1, depth_2, depth_3, depth_4, pages]:
            ls.append(np.NaN)
    elif 'check' not in pandan:
        if len(pandan) == 1:
#             print('len1')
            for num, line in enumerate(text):
                d,p,e = split_pn(line,e) ## tnwjd
                depth_1.append(d)
                depth_2.append(np.NaN)
                depth_3.append(np.NaN)
                depth_4.append(np.NaN)
                etc.append(e) # 문제 없음
                pages.append(p)
                isbn_ls.append(code)
                main_title.append(title)
        elif len(pandan) == 2:
            idx_plus(pandan, text, code, title, e)
        elif len(pandan) == 3:
            idx_plus(pandan, text, code, title, e)
        elif len(pandan) >= 4:
            idx_plus(pandan, text, code, title,e)
       


In [5]:
def idx_plus(pandan, text, code, title,e):
    d1,d2,d3,d4 = '','','',''
    for num, line in enumerate(text):
        a = re.sub(r'[^(Ⅹ|Ⅸ|Ⅳ|Ⅴ?Ⅰ|Ⅱ|Ⅲ)]','', line) 
        d,p,e = split_pn(line,e) #tnwjd
        if d[0].isnumeric() == True and d[1] in ['부', '절','편','강']: # 숫자분류
            check = 'num'
        elif d[0].isnumeric() == True  and d[1] not in ['부', '절','편','강'] and a == '': # 아라비아
            check = 'ara'
        elif d[0].isnumeric() == True and a != '': # roma
            check = 'roma'
        elif d[0].isnumeric() != True:
            base = re.sub(r'[^a-zA-Z가-힣]','',line.lower())
            target = ex.match(base)
            if target:
                check = target.group()
                if base.count(target.group())>1:
                    k = 'over'
                    e = etc_check(e,k)
            else:
                pass
            
        if type(pandan[0]) == list:
            try:
                ii = pandan.index(check)
                if ii == 0:
                    d1 = d
                    d2,d3,d4 = '','',''
                elif ii == 1:
                    d2 = d
                    d3,d4 = '',''
                elif ii == 2:
                    d3 = d
                    d4 = ''
                else:   # 4뎁스 이후꺼 삭제시 이걸로 조절
                    d4 = d
            except:
                d1 = d
            for ls in [0,1,2,3]:
                d_append(ls,d1,d2,d3,d4)
            etc.append(e)
            pages.append(p)
            isbn_ls.append(code)
            main_title.append(title)
            
        elif type(pandan[1]) == list:  # 이부분 추가됨
            try:
                ii = pandan.index(check)
                if ii == 0:
                    d1 = d
                    d2,d3,d4 = '','',''
                elif ii == 1:
                    d2 = d
                    d3,d4 = '',''
                elif ii == 2:
                    d3 = d
                    d4 = ''
                else:   # 4뎁스 이후꺼 삭제시 이걸로 조절
                    d4 = d
            except:
                d2 = d
            for ls in [0,1,2,3]:
                d_append(ls,d1,d2,d3,d4)
            etc.append(e)
            pages.append(p)
            isbn_ls.append(code)
            main_title.append(title)
            
        else:
            print('store3')
            ii = pandan.index(check)
            ii = pandan.index(check)
            if ii == 0:
                d1 = d
                d2,d3,d4 = '','',''
            elif ii == 1:
                d2 = d
                d3,d4 = '',''
            elif ii == 2:
                d3 = d
                d4 = ''
            else:   # 4뎁스 이후꺼 삭제시 이걸로 조절
                d4 = d
            for ls in [0,1,2,3]:
                d_append(ls,d1,d2,d3,d4)
            etc.append(e)
            pages.append(p)
            isbn_ls.append(code)
            main_title.append(title)
        print(d1,d2,d3,d4,'store')
#         store(pandan, check, d, p, code, title,d1,d2,d3,d4) # if_else문으로 check를 받고 시행하려고
        

In [6]:
def d_append(ls,d1,d2,d3,d4):
    if ls == 0:
        depth_dict[f'{ls}'].append(d1)
    elif ls == 1:
        depth_dict[f'{ls}'].append(d2)
    elif ls == 2:
        depth_dict[f'{ls}'].append(d3)
    elif ls == 3:
        depth_dict[f'{ls}'].append(d4)
    print('d_append')

# making

In [7]:
# def check_pandan(pd_l, ck_l):
#     if len(pd_l) == 1:
#         e = 0
#     else:
#         print('cp on')
#         check = ''.join(ck_l).split(pd_l[0])
#         pp = ''.join(pd_l[1:])
#         if len(check) == 2:
#             e = 0
#         elif len(check) >= 3:
#             if check[1][:len(pp)] == check[2][:len(pp)]:
#                 e = 0 # 문제 없음
#             elif check[1][:len(pp)] != check[2][:len(pp)]:
#                 e = 'd_check'
#             else:
#                 e = 3 # 구분주려고 임시
#                 print('???checkpandan')
#         elif len(check) == 1:
#             e = 2
#         else:
#             e = 3 # 구분주려고 임시
#             print('??checkpandan')
#     return e
            
        

In [8]:
def pandan_sequence(pd_l, ck_l):
    all_ls = ' '.join(ck_l).split(pd_l[0])
    a_ls = []
    print('pas')
    for a in all_ls[1:]:
    #     if c not in le:
        x = []
        for aa in a.split(' '):
            if aa not in x:
                x.append(aa)
        a_ls.append(x)
    if len(list(set(tuple(a) for a in a_ls))) > 1:
        k = 'seq'
        print('pas2')
        e = etc_check(e, k)
        print('pas333')
    elif len(list(set(tuple(a) for a in a_ls))) == 1:
        k = 0
        e = etc_check(e, k)
    else:
        k = 0
        e = etc_check(e, k)
        print('????pandansequence')
    return e

In [19]:
dict_error['pn']

'page_check'

In [9]:
def etc_check(e, k):
    print('etc')
    if e == 0 and k != 0:
        print('etc1')
        e = dict_error[k]
    elif e == 0 and k == 0:
        e = 0
    elif e != 0 and k == 0:
        e = e
    elif e != 0 and k != 0:
        if e == 'double_check':
            e = 'three_check'
        else:
            e = 'double_check'
    print('etcs2')
    return e
    
# for p in range(len(a)):
#     if type(p) != float:
#         if len(p) < 4:
#             e = 0
#         else:
#             k = 'pn'
#             e = etc_check(e, p, k)

# data loading & variable

In [10]:
# data  police_book_idx_unique
data = pd.read_excel("C:\police_data\index_one_book.xlsx")
# 변수 설정
isbn, tit, text = data['isbn'], data['main_title'], data['index']

# 저장될 리스트
isbn_ls, main_title, depth_1, depth_2, depth_3, depth_4, etc, pages = [],[],[],[],[],[],[],[]

# depth정보를 저장할 변수
d1,d2,d3,d4 = '','','',''
# 구분자 리스트
pandan = []
# 구분자, 교재명
check, title = '',''
# isbn, 페이지
code, p = 0,0

In [11]:
# 지정 구분자
ex = re.compile("제절|제편|제강|제항|제장|제회|제부|제과|제관|강|목|장|항|회|부|사례|총론|형법총론|헌법총론|각론|형법각론|헌법각론|부록|키워드|최근|실전|모의|문제|정답|해설|차례|전범위|기출|참고|판례|핵심|part|chapter|day|section|theme|case|keyword|point|q|question|contents")

In [12]:
# 편한 저장을 위해 딕셔너리
depth_dict = {'0':depth_1,'1':depth_2,'2':depth_3,'3':depth_4}
d_dict = {'0':d1,'1':d2,'2':d3,'3':d4} 
dict_error = {'pn':'page_check', 'over':'over_check','seq' : 'sequence_check'}
# 실행
for i in range(len(text)):
    try:
        tet = make_ls(text, i)
        clear_idx(isbn, tit, tet, i, ex)
    except:
#         print('i')
        isbn_ls.append(isbn[i])
        main_title.append(tit[i])
        etc.append(2)  # 데이터가 존재하지 않는 경우
        for k in [depth_1, depth_2,depth_3,depth_4,pages]:
            k.append(np.NaN)

iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
pas2
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
pas2
iii
pas
pas2
iii
pas
iii
pas
pas2
iii
pas
pas2
iii
pas
iii
pas
pas2
iii
pas
iii
pas
iii
pas
iii
pas
pas2
iii
pas
pas2
iii
pas
pas2
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
pas2
iii
pas
pas2
iii
pas
pas2
iii
pas
pas2
iii
pas
pas2
iii
pas
iii
pas
iii
pas
iii
pas
pas2
iii
pas
iii
pas
pas2
iii
pas
iii
pas
pas2
iii
pas
iii
pas
iii
pas
pas2
iii
pas
iii
pas
pas2
iii
pas
pas2
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
pas2
iii
pas
pas2
iii
pas
pas2
iii
pas
pas2
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
pas2
iii
pas
iii
pas
iii
pas
pas2
iii
pas
iii
pas
pas2
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
pas2
iii
pas
iii
pas
pas2
iii
pas
iii
pas
iii
pas
pas2
iii
pas
pas2
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
pas2
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
iii
pas
pas2
iii
pas
pas2
iii
pa

In [13]:
len(main_title), len(depth_1)

(176, 176)

In [14]:
df_ls = pd.DataFrame({'isbn':isbn_ls, 'main_title':main_title,'depth_1':depth_1, 'depth_2':depth_2,\
                      'depth_3':depth_3, 'depth_4': depth_4,'pages':pages,'etc':etc})

In [15]:
# df_ls.to_excel('C:/police_data/06_30_test2.xlsx',encoding = 'utf-8-sig', index=False)

In [16]:
df_ls.loc[:,'depth_2'].unique()

array([nan])

In [17]:
df_ls

,isbn,main_title,depth_1,depth_2,depth_3,depth_4,pages,etc
0,9791166184604,2023 경찰 헌법집중 기출해설,NaN,NaN,NaN,NaN,NaN,2
1,9791197865367,한 권으로 끝내는 경찰범죄학(기본서),NaN,NaN,NaN,NaN,NaN,2
2,9791169873437,박문각 경찰 박용증 아두스 경찰학 실전동형 10회분,NaN,NaN,NaN,NaN,NaN,2
3,9791192989082,신간2024 최근 14년간 형사소송법 기출총정리,NaN,NaN,NaN,NaN,NaN,2
4,9788952645432,2023 해양경찰 기관술(학) 기출문제집,NaN,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...
171,9791166184871,경찰 헌법도약 기출지문 OX,NaN,NaN,NaN,NaN,NaN,2
172,9791168809055,2023 해커스경찰 박철한 경찰헌법 실전동형모의고사,NaN,NaN,NaN,NaN,NaN,2
173,9791168809154,2023 해커스경찰 갓대환 형사법 진도별 문제풀이 1000제 1차 시험 대비,NaN,NaN,NaN,NaN,NaN,2
174,9791168808713,2023 해커스경찰 갓대환 형사법 기본서 1: 형법,NaN,NaN,NaN,NaN,NaN,2


In [18]:
text = '제항어쩌구저쩌구제항두번쨰'
# target = '제항'
text.count(target)

NameError: name 'target' is not defined